# Przewidywanie przekładni

- WOS 174/L  PRZEKŁADNIA 13,5LHR32400  praca od 21.05.2019 do 02.10.2020,
- WOS 176/L  PRZEKŁADNIA 13,5LHR32400  praca od 20.08.2019 do 28.01.2021,
- WOS 177/L  PRZEKŁADNIA 13,5LHR32400  praca od 09.01.2020 do 26.02.2021,
- WOS 179/L PRZEKŁADNIA 13,5LHR32400  praca od 14.02.2020 do 05.05.2021

In [2]:
import pandas as pd

import datetime
import pyarrow

## WOS 174/L

In [3]:
df = pd.read_parquet('../data/WOS_176L_2020_09.parquet', engine='pyarrow')
df.head(3)

,maszyna,data,czas,trnlup,groilp,selgear,speed,breakp,engtps,trnaut,...,fuelus,engoilp,intakep,hyddrv,engcoolt,intaket,groilt,hydoilt,tempin,enghours
0,KLDSMG_WOS___176L,2020/09/01,00:00:00.000,0.0,1728.0,2.0,0.0,0.0,61.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KLDSMG_WOS___176L,2020/09/01,00:00:01.000,0.0,1856.0,2.0,3.0,0.0,96.0,0.0,...,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KLDSMG_WOS___176L,2020/09/01,00:00:02.000,1.0,1904.0,2.0,6.0,0.0,96.0,0.0,...,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_wos174 = pd.DataFrame(columns=df.columns)

for file_date in ['2020_10','2020_09','2020_11','2020_12','2021_01']:
    df_tmp = pd.read_parquet('../data/WOS_176L_' + file_date + '.parquet', engine='pyarrow')
    df_wos174 = pd.concat([df_wos174, df_tmp], axis=0)

In [5]:
df_wos174_copy = df_wos174.copy()

In [6]:
df_wos174 = df_wos174_copy.copy()

#zmienna wróżka; bo jest narastająca, zatem jak blisko konkretnej daty się psują to jest tu zakodowane info o statusie
df_wos174 = df_wos174.drop(columns=['enghours'])

#mało wartości
df_wos174 = df_wos174.drop(columns=['trnaut'])

#cleaning if all values are NaN
#https://stackoverflow.com/questions/13413590/how-to-drop-rows-of-pandas-dataframe-whose-value-in-a-certain-column-is-nan
df_wos174 = df_wos174.dropna(thresh=len(df_wos174.columns)-3) 

#datatime column
df_wos174['dt'] = pd.to_datetime(df_wos174['data'] + ' ' + df_wos174['czas'])
df['data'] = df['data'].astype('datetime64[ns]')

#sorting descending by date
df_wos174 = df_wos174.sort_values(by=['dt'], ascending = False).reset_index(drop=True)

#delete observations after failure
df_wos174 = df_wos174[df_wos174['dt']<'2021-01-28']

#prepare aggregats
df_wos174_1min = df_wos174.resample('1min', on='dt').mean().reset_index().set_index('dt').fillna(method='bfill')
df_wos174_1hour = df_wos174.resample('1H', on='dt').mean().reset_index().set_index('dt').fillna(method='bfill')
df_wos174_1day = df_wos174.resample('1D', on='dt').mean().reset_index().set_index('dt').fillna(method='bfill')

In [7]:
# tworzymy cechy 6h wcześniej, 12h wcześniej 24h wcześniej 48h wcześniej, 7dni wcześniej
# i agregaty mean (jak pyknie dodamy min i max)

df_tmp_6h = df_wos174_1hour.rolling(6).mean()
df_tmp_6h.columns = ['avg6_'+ col for col in list(df_tmp_6h.columns)]

df_tmp_12h = df_wos174_1hour.rolling(12).mean()
df_tmp_12h.columns = ['avg12_'+ col for col in list(df_tmp_12h.columns)]

df_tmp_24h = df_wos174_1hour.rolling(24).mean()
df_tmp_24h.columns = ['avg24_'+ col for col in list(df_tmp_24h.columns)]

df_tmp_48h = df_wos174_1hour.rolling(48).mean()
df_tmp_48h.columns = ['avg48_'+ col for col in list(df_tmp_48h.columns)]

df_tmp_168h = df_wos174_1hour.rolling(168).mean()
df_tmp_168h.columns = ['avg168_'+ col for col in list(df_tmp_168h.columns)]

In [8]:
df_wos174_1hour_hist = pd.concat([df_wos174_1hour, df_tmp_6h, df_tmp_12h, df_tmp_24h, df_tmp_48h, df_tmp_168h],axis=1)

In [9]:
df_wos174_1hour_hist = df_wos174_1hour_hist.reset_index()
# status good / bad. If 2 weeks before failture then bad else good

df_wos174_1hour_hist['status'] = df_wos174_1hour_hist['dt'].apply(lambda x: 0 if x < datetime.datetime(2021,1,28)-datetime.timedelta(days=14) else 1)

# buffor - 2 weeks between good and bads
df_wos174_1hour_hist['buffor'] = df_wos174_1hour_hist['dt'].apply(lambda x: 1 if x < datetime.datetime(2021,1,28)-datetime.timedelta(days=30) else 0)

# not longer then 3 months before failure
df_wos174_1hour_hist['notLonger'] = df_wos174_1hour_hist['dt'].apply(lambda x: 0 if x < datetime.datetime(2021,1,28)-datetime.timedelta(days=90) else 1)


In [10]:
df_wos174_1hour_hist_final = pd.concat([df_wos174_1hour_hist[df_wos174_1hour_hist['status']==1],
                                        df_wos174_1hour_hist[(df_wos174_1hour_hist['status']==0) & (df_wos174_1hour_hist['buffor']==1) & (df_wos174_1hour_hist['notLonger']==1)]
                                       ], axis=0)

print(f'shape: {df_wos174_1hour_hist_final.shape}')

df_wos174_1hour_hist_final.status.value_counts()

shape: (1776, 112)


0    1440
1     336
Name: status, dtype: int64

In [11]:
#df_wos174_1hour_hist_final = df_wos174_1hour_hist_final.drop(columns=['index']).iloc[:,1:-2].reset_index(drop=True)
df_wos174_1hour_hist_final = df_wos174_1hour_hist_final.iloc[:,1:-2].reset_index(drop=True)
df_wos174_1hour_hist_final.head(3)

,trnlup,groilp,selgear,speed,breakp,engtps,trnbps,hydoilp,engrpm,fuelus,...,avg168_fuelus,avg168_engoilp,avg168_intakep,avg168_hyddrv,avg168_engcoolt,avg168_intaket,avg168_groilt,avg168_hydoilt,avg168_tempin,status
0,0.095238,1161.142857,0.357143,1.125,3192.857143,11.47619,25.047619,0.806905,637.357143,3.952381,...,4.810756,157.157271,20.668836,0.000053,63.491798,36.034438,37.048351,36.079331,24.175251,1
1,0.095238,1161.142857,0.357143,1.125,3192.857143,11.47619,25.047619,0.806905,637.357143,3.952381,...,4.731851,156.370126,20.184418,0.000053,63.234301,35.874053,36.757542,35.840534,24.209526,1
2,0.095238,1161.142857,0.357143,1.125,3192.857143,11.47619,25.047619,0.806905,637.357143,3.952381,...,4.684296,155.579309,19.947422,0.000053,62.981441,35.725771,36.477348,35.607417,24.230260,1


In [12]:
df_wos174_1hour_hist_final.to_csv('../data/df_wos176.csv', sep="|", index=False)